In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

## Assess ACS Data Tidiness

In [2]:
acs_07 = pd.read_csv('resources/data/ACS_07.csv')
acs_07.head()

,GEO.display-label,HD01_VD01,HD01_VD02,HD02_VD02,HD01_VD03,HD02_VD03,HD01_VD04,HD02_VD04,HD01_VD05,HD02_VD05,HD01_VD06,HD02_VD06,HD01_VD07,HD02_VD07,HD01_VD08,HD02_VD08,HD01_VD09,HD02_VD09,HD01_VD10,HD02_VD10
0,Alabama,4627851,3254107,5821,1212118,5691,20690,1954,44561,3209,1518,1310,34689,3695,60168,4763,5101,1570,55067,4495
1,Alaska,683478,465234,2534,25821,2172,92058,3176,32936,1975,4978,1185,15531,2952,46920,4037,1506,731,45414,3878
2,Arizona,6338755,4841990,21733,224991,5520,281096,5218,151414,3474,7701,1101,677188,21346,154375,9650,54418,5979,99957,7714
3,Arkansas,2834797,2226758,5666,442279,4286,17425,2101,33933,2991,2274,727,63856,5827,48272,4478,4287,1805,43985,3937
4,California,36553215,22026736,61328,2263363,12575,266051,10336,4511407,14112,126345,4508,6096927,64388,1262386,26412,407953,16135,854433,19033


Data is not in a tidy format - need year, estimate (HD01), margin of error (HD02), and count_type columns. 

### Clean
- Create `year` column with relevant year for file
- Modify column names for ease of use with `pd.wide_to_long()`
- Automate creation of df for files from 2007 to 2016

In [3]:
acs_07['year'] = 2007
acs_07.head()

,GEO.display-label,HD01_VD01,HD01_VD02,HD02_VD02,HD01_VD03,HD02_VD03,HD01_VD04,HD02_VD04,HD01_VD05,HD02_VD05,...,HD02_VD06,HD01_VD07,HD02_VD07,HD01_VD08,HD02_VD08,HD01_VD09,HD02_VD09,HD01_VD10,HD02_VD10,year
0,Alabama,4627851,3254107,5821,1212118,5691,20690,1954,44561,3209,...,1310,34689,3695,60168,4763,5101,1570,55067,4495,2007
1,Alaska,683478,465234,2534,25821,2172,92058,3176,32936,1975,...,1185,15531,2952,46920,4037,1506,731,45414,3878,2007
2,Arizona,6338755,4841990,21733,224991,5520,281096,5218,151414,3474,...,1101,677188,21346,154375,9650,54418,5979,99957,7714,2007
3,Arkansas,2834797,2226758,5666,442279,4286,17425,2101,33933,2991,...,727,63856,5827,48272,4478,4287,1805,43985,3937,2007
4,California,36553215,22026736,61328,2263363,12575,266051,10336,4511407,14112,...,4508,6096927,64388,1262386,26412,407953,16135,854433,19033,2007


In [4]:
col = list(acs_07.columns)

In [5]:
col = [name.replace('VD0', '') if name[:3] == 'HD0' else name for name in col]

In [6]:
col = [name.replace('VD', '') if name[:3] == 'HD0' else name for name in col]

In [7]:
col

['GEO.display-label',
 'HD01_1',
 'HD01_2',
 'HD02_2',
 'HD01_3',
 'HD02_3',
 'HD01_4',
 'HD02_4',
 'HD01_5',
 'HD02_5',
 'HD01_6',
 'HD02_6',
 'HD01_7',
 'HD02_7',
 'HD01_8',
 'HD02_8',
 'HD01_9',
 'HD02_9',
 'HD01_10',
 'HD02_10',
 'year']

In [8]:
acs_07.columns = col

In [9]:
df = pd.wide_to_long(acs_07, stubnames=['HD01', 'HD02'], i=['GEO.display-label', 'year'], j='count_type', sep='_').reset_index()
df.head()

,GEO.display-label,year,count_type,HD01,HD02
0,Alabama,2007,1,4627851,NaN
1,Alabama,2007,10,55067,4495.0
2,Alabama,2007,2,3254107,5821.0
3,Alabama,2007,3,1212118,5691.0
4,Alabama,2007,4,20690,1954.0


In [10]:
folder_name = 'resources/data/'
df = pd.DataFrame([], columns=df.columns)
year = 2007
for file in os.listdir(folder_name):
    if file[:3] == 'ACS':
        acs = pd.read_csv(folder_name + file)
        acs['year'] = year
        acs.columns = col
        acs = pd.wide_to_long(acs, stubnames=['HD01', 'HD02'], i=['GEO.display-label', 'year'], j='count_type', sep='_').reset_index()
        df = pd.concat([df, acs])
        year += 1

## Assess ACS Cleanliness

In [11]:
df.head()

,GEO.display-label,year,count_type,HD01,HD02
0,Alabama,2007,1,4627851,NaN
1,Alabama,2007,10,55067,4495.0
2,Alabama,2007,2,3254107,5821.0
3,Alabama,2007,3,1212118,5691.0
4,Alabama,2007,4,20690,1954.0


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5100 entries, 0 to 509
Data columns (total 5 columns):
GEO.display-label    5100 non-null object
year                 5100 non-null object
count_type           5100 non-null object
HD01                 5100 non-null object
HD02                 4590 non-null float64
dtypes: float64(1), object(4)
memory usage: 239.1+ KB


In [13]:
df.nunique()

GEO.display-label      51
year                   10
count_type             10
HD01                 5017
HD02                 3818
dtype: int64

In [14]:
df.year.unique()

array([2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016],
      dtype=object)

- `count_type` should be converted to descriptive names
- Column names should be changed to more descriptive names
- `HD01` should be changed to `int`
- Need state abbreviation for joining with other data

## Clean

#### Column names
Column names should be changed to 'state_name', 'year', 'count_type', 'estimate', 'margin_of_error'

In [20]:
cols = ['state_name', 'year', 'count_type', 'estimate', 'margin_of_error']
df.columns = cols
df.head()

,state_name,year,count_type,estimate,margin_of_error
0,Alabama,2007,total,4627851,NaN
1,Alabama,2007,10,55067,4495.0
2,Alabama,2007,2,3254107,5821.0
3,Alabama,2007,3,1212118,5691.0
4,Alabama,2007,4,20690,1954.0


#### Descriptive names for `count_type`
Use `.replace()` to convert names

In [21]:
df = df.replace({
    'count_type': {
        '1': 'total',
        '2': 'white',
        '3': 'black_aa',
        '4': 'am_indian_native',
        '5': 'asian',
        '6': 'hawaiian_islander',
        '7': 'other',
        '8': 'two_or_more',
        '9': 'two_or_more_inculding_other',
        '10': 'two_or_more_exc_other_or_three_plus'
    }
})  

In [22]:
df.head()

,state_name,year,count_type,estimate,margin_of_error
0,Alabama,2007,total,4627851,NaN
1,Alabama,2007,two_or_more_exc_other_or_three_plus,55067,4495.0
2,Alabama,2007,white,3254107,5821.0
3,Alabama,2007,black_aa,1212118,5691.0
4,Alabama,2007,am_indian_native,20690,1954.0


In [23]:
df.count_type.unique()

array(['total', 'two_or_more_exc_other_or_three_plus', 'white',
       'black_aa', 'am_indian_native', 'asian', 'hawaiian_islander',
       'other', 'two_or_more', 'two_or_more_inculding_other'],
      dtype=object)

#### Convert `estimate` to `int`
Covert estimate to `int` with `.astype(int)`

In [24]:
df.estimate = df.estimate.astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5100 entries, 0 to 509
Data columns (total 5 columns):
state_name         5100 non-null object
year               5100 non-null object
count_type         5100 non-null object
estimate           5100 non-null int32
margin_of_error    4590 non-null float64
dtypes: float64(1), int32(1), object(3)
memory usage: 219.1+ KB


#### Add state abbreviations
- Import state names data
- Join to df
- Drop `state_names`

In [27]:
states = pd.read_csv('resources/data/states.csv')
states.head()

,Unnamed: 0,abbr,name
0,0,AL,Alabama
1,1,AK,Alaska
2,2,AZ,Arizona
3,3,AR,Arkansas
4,4,CA,California


In [28]:
states.drop('Unnamed: 0', axis=1, inplace=True)
states.head()

,abbr,name
0,AL,Alabama
1,AK,Alaska
2,AZ,Arizona
3,AR,Arkansas
4,CA,California


In [36]:
states.to_csv('resources/data/states.csv', index=False)

In [29]:
df = df.merge(states, how='left', left_on='state_name', right_on='name')
df.head()

,state_name,year,count_type,estimate,margin_of_error,abbr,name
0,Alabama,2007,total,4627851,NaN,AL,Alabama
1,Alabama,2007,two_or_more_exc_other_or_three_plus,55067,4495.0,AL,Alabama
2,Alabama,2007,white,3254107,5821.0,AL,Alabama
3,Alabama,2007,black_aa,1212118,5691.0,AL,Alabama
4,Alabama,2007,am_indian_native,20690,1954.0,AL,Alabama


In [32]:
df.drop(['state_name', 'name'], axis=1, inplace=True)
df.head()

,year,count_type,estimate,margin_of_error,abbr
0,2007,total,4627851,NaN,AL
1,2007,two_or_more_exc_other_or_three_plus,55067,4495.0,AL
2,2007,white,3254107,5821.0,AL
3,2007,black_aa,1212118,5691.0,AL
4,2007,am_indian_native,20690,1954.0,AL


In [33]:
df.rename(columns={'abbr': 'state'}, inplace=True)
df.head()

,year,count_type,estimate,margin_of_error,state
0,2007,total,4627851,NaN,AL
1,2007,two_or_more_exc_other_or_three_plus,55067,4495.0,AL
2,2007,white,3254107,5821.0,AL
3,2007,black_aa,1212118,5691.0,AL
4,2007,am_indian_native,20690,1954.0,AL


In [35]:
df.to_csv('datasets/population_acs.csv', index=False)

## EDA

**Confirm the arrangement of the `count_type` categories.**

In [37]:
pop_07 = df.query('year == 2007')
pop_07.head()

,year,count_type,estimate,margin_of_error,state
0,2007,total,4627851,NaN,AL
1,2007,two_or_more_exc_other_or_three_plus,55067,4495.0,AL
2,2007,white,3254107,5821.0,AL
3,2007,black_aa,1212118,5691.0,AL
4,2007,am_indian_native,20690,1954.0,AL


In [41]:
AL_pop_07 = pop_07.query('state == "AL"')
AL_pop_07

,year,count_type,estimate,margin_of_error,state
0,2007,total,4627851,NaN,AL
1,2007,two_or_more_exc_other_or_three_plus,55067,4495.0,AL
2,2007,white,3254107,5821.0,AL
3,2007,black_aa,1212118,5691.0,AL
4,2007,am_indian_native,20690,1954.0,AL
5,2007,asian,44561,3209.0,AL
6,2007,hawaiian_islander,1518,1310.0,AL
7,2007,other,34689,3695.0,AL
8,2007,two_or_more,60168,4763.0,AL
9,2007,two_or_more_inculding_other,5101,1570.0,AL


In [42]:
AL_pop_07.estimate[1:].sum()

4688019

The sum of all other categories is more than the total, so there is some duplicate counting.

In [40]:
4688019 - 4627851

60168

In [43]:
55067 + 5101

60168

It appears that `two_or_more` is a recount combination of `two_or_more_exc_other_or_three_plus` and `two_or_more_inculding_other`.

For the purposes of this work, this breakdown does not seem critical, therefore, we will retain `two_or_more` and drop the other two in the main df.

In [57]:
df = df[(df.count_type != 'two_or_more_exc_other_or_three_plus') & \
        (df.count_type != 'two_or_more_inculding_other')]
df.head()

,year,count_type,estimate,margin_of_error,state
0,2007,total,4627851,NaN,AL
2,2007,white,3254107,5821.0,AL
3,2007,black_aa,1212118,5691.0,AL
4,2007,am_indian_native,20690,1954.0,AL
5,2007,asian,44561,3209.0,AL


In [58]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4080 entries, 0 to 5098
Data columns (total 5 columns):
year               4080 non-null object
count_type         4080 non-null object
estimate           4080 non-null int32
margin_of_error    3570 non-null float64
state              4000 non-null object
dtypes: float64(1), int32(1), object(3)
memory usage: 175.3+ KB


In [59]:
df.to_csv('datasets/population_acs.csv', index=False)

**Compare to Census data**

In [60]:
census_pop = pd.read_csv('datasets/population.csv')
census_pop.head()

,state,year,population
0,Alabama,2010,4785579
1,Alaska,2010,714015
2,Arizona,2010,6407002
3,Arkansas,2010,2921737
4,California,2010,37327690


In [64]:
df.query('year == 2010 & count_type == "total"').head()

,year,count_type,estimate,margin_of_error,state
1530,2010,total,4785298,NaN,AL
1540,2010,total,713985,NaN,AK
1550,2010,total,6413737,NaN,AZ
1560,2010,total,2921606,NaN,AR
1570,2010,total,37349363,NaN,CA


It appears that the ACS data is very similar to the original population data. Because the ACS data is associated with many other metrics that could be useful, it makes sense to retain this as the primary data source for population. 

It is noted that ACS data is not currently available for 2017.